## First look of the CALIPSO data
(from Haochi)

The files contain, binned on temperature.:
- supercooled-count
- total count

Variants of the datasets:
- 2009_2023: all the years form 2008 to 2023
- 2022: only 2022
Months:
- FM: February - March
- JFMA: January - April
- DJMA: December - April

The region is 70-78.5 degrees and longitudes 6 to 25, ocean only. Data divided into:
- whole region
- North part
- south part

In [ ]:
# --- Functions
import xarray as xr # read netcdf-files

In [ ]:
# --- Paths
cal_path = '/home/ninalar/Documents/MC2/Calipso_fromHaochi/'

# calipso file names:
files = ['SLF_2009_2023_JFMA_south.nc',
         'SLF_2022_JFMA_north.nc',
         'SLF_top_2009_2023_FM_whole.nc',
         'SLF_top_2022_FM_south.nc',
         'SLF_2009_2023_DJMFA_north.nc',
         'SLF_2009_2023_JFMA_whole.nc  '
         'SLF_2022_JFMA_south.nc',
         'SLF_top_2009_2023_JFMA_north.nc',
         'SLF_top_2022_FM_whole.nc',
         'SLF_2009_2023_DJMFA_south.nc',
         'SLF_2022_DJMFA_north.nc',
         'SLF_2022_JFMA_whole.nc',
         'SLF_top_2009_2023_JFMA_south.nc',
         'SLF_top_2022_JFMA_north.nc',
         'SLF_2009_2023_DJMFA_whole.nc',
         'SLF_2022_DJMFA_south.nc',
         'SLF_top_2009_2023_DJMFA_north.nc',
         'SLF_top_2009_2023_JFMA_whole.nc',
         'SLF_top_2022_JFMA_south.nc',
         'SLF_2009_2023_FM_north.nc',
         'SLF_2022_DJMFA_whole.nc',
         'SLF_top_2009_2023_DJMFA_south.nc',
         'SLF_top_2022_DJMFA_north.nc',
         'SLF_top_2022_JFMA_whole.nc',
         'SLF_2009_2023_FM_south.nc',
         'SLF_2022_FM_north.nc',
         'SLF_top_2009_2023_DJMFA_whole.nc',
         'SLF_top_2022_DJMFA_south.nc',
         'SLF_2009_2023_FM_whole.nc',
         'SLF_2022_FM_south.nc',
         'SLF_top_2009_2023_FM_north.nc',
         'SLF_top_2022_DJMFA_whole.nc',
         'SLF_2009_2023_JFMA_north.nc',
         'SLF_2022_FM_whole.nc',
         'SLF_top_2009_2023_FM_south.nc',
         'SLF_top_2022_FM_north.nc']

print(cal_path + files[0])

/home/ninalar/Documents/MC2/Calipso_fromHaochi/SLF_2009_2023_JFMA_south.nc


In [5]:
# first look at data
test_file = cal_path + files[0]

test_ds = xr.open_dataset(test_file)
test_ds

<xarray.Dataset> Size: 7kB
Dimensions:      (year: 15, temperature: 28)
Coordinates:
  * year         (year) int64 120B 2009 2010 2011 2012 ... 2020 2021 2022 2023
  * temperature  (temperature) float64 224B 224.0 226.0 228.0 ... 276.0 278.0
Data variables:
    count        (year, temperature) int64 3kB ...
    slf          (year, temperature) int64 3kB ...

In [ ]:
# plotting the 2Dhist of the SLF
# function for plotting the margplot with temperature on x-axis and SLF on Y-axis

def slf_marg_plot(df, title, obs_cat,filestr = ""):
    import matplotlib.gridspec as gridspec

    # preparing for histogram and 2d histogram
    temperature = df['Temperature (Celsius)'].values
    slf = df['SLF'].values

    # Define bins and range
    x_bins = 20
    y_bins = 20
    x_range = [-40, 0]
    y_range = [0, 100]

    # Compute the 2D histogram with numpy histogram2d to get the normalization correct
    hist, xedges, yedges = np.histogram2d(temperature, slf, 
                                       bins=[x_bins, y_bins], 
                                       range=[x_range, y_range], 
                                       density=True)
    
    fig = plt.figure(figsize=(12,8))

    plt.rcParams['font.size'] = 18
    
    gs = gridspec.GridSpec(3, 4, wspace = 0, hspace = 0)
    ax_main = plt.subplot(gs[1:3, 1:3])
    ax_xDist = plt.subplot(gs[0, 1:3],sharex=ax_main)
    ax_yDist = plt.subplot(gs[1:3, 3],sharey=ax_main)
    ax_cb = plt.subplot(gs[1:3,0],sharey=ax_main)
    ax_cb.axis('off')
    ax_text = plt.subplot(gs[0,3])
    ax_text.axis('off')
    plots.letter_annotation(ax_text, 0.3, 0.3, f'tot. # of \n{obs_cat} obs.: \n{len(df)}')
        
    hh = ax_main.imshow(hist.T, origin='lower', cmap=plt.cm.Blues, 
                     extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]],
                     aspect='auto')
    ax_main.set_ylabel('SLF %')
    ax_main.set_xlabel('Temperature ($\mathregular{\degree C}$)')
    
    
    ax_xDist.hist(temperature,bins=x_bins, align='left', density = True) # Density = True means normalized
    ax_xDist.set(ylabel='norm. count')
    ax_xDist.tick_params(top=True, labeltop=True, bottom=False, labelbottom=False)

    old_ticks=ax_xDist.get_yticks()
    new_ticks=[x for x in old_ticks if x!=0]
    ax_xDist.set_yticks(new_ticks)
    
    ax_yDist.hist(slf,bins=y_bins,orientation='horizontal', density = True)
    ax_yDist.set(xlabel='norm. count')
    ax_yDist.yaxis.tick_right()
    old_ticks=ax_yDist.get_xticks() # removing 0 from ticks (improves readability)
    new_ticks=[x for x in old_ticks if x!=0]
    ax_yDist.set_xticks(new_ticks)
    
    cbar = plt.colorbar(hh, ax = ax_cb, location='left', orientation='vertical')
    cbar.set_label('norm. # of SLFs per Temp', rotation=90)
    
    fig.suptitle(title, fontsize = 'large')
    plt.show

    if filestr != '':
        plt.savefig(filestr)
    
    return(fig)